In [2]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection


Cloning into 'mmdetection'...
remote: Enumerating objects: 38023, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 38023 (delta 0), reused 1 (delta 0), pack-reused 38019 (from 1)
Receiving objects: 100% (38023/38023), 63.18 MiB | 17.39 MiB/s, done.
Resolving deltas: 100% (26232/26232), done.
/content/mmdetection


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install mmdet
!pip install -U openmim
!mim install mmdet
!pip install mmcv==2.0.0rc4

In [ ]:
# Import required libraries
import os
import mmcv
from mmdet.apis import init_detector, train_detector
from mmdet.datasets import build_dataset
from mmdet.utils import collect_env, get_root_logger

# Define paths for the dataset
dataset_root = '/content/drive/MyDrive/Count_AI_task/Aquarium_Combined.v2-raw-1024.coco-mmdetection'
train_ann_file = os.path.join(dataset_root, 'train/_annotations.coco.json')
valid_ann_file = os.path.join(dataset_root, 'valid/_annotations.coco.json')
test_ann_file = os.path.join(dataset_root, 'test/_annotations.coco.json')

# Define dataset configuration
data_config = dict(
    train=dict(
        type='CocoDataset',
        ann_file=train_ann_file,
        img_prefix=os.path.join(dataset_root, 'train/images/'),
        classes=('species1', 'species2', 'species3')  # Update with actual class names
    ),
    val=dict(
        type='CocoDataset',
        ann_file=valid_ann_file,
        img_prefix=os.path.join(dataset_root, 'valid/images/'),
        classes=('species1', 'species2', 'species3')
    ),
    test=dict(
        type='CocoDataset',
        ann_file=test_ann_file,
        img_prefix=os.path.join(dataset_root, 'test/images/'),
        classes=('species1', 'species2', 'species3')
    )
)

# Load model configuration from MMDetection model zoo (e.g., Faster R-CNN)
config_file = 'configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'  # Example model config
model = init_detector(config_file, device='cuda:0')

# Modify the model's configuration for your dataset
model.cfg.data.train = data_config['train']
model.cfg.data.val = data_config['val']
model.cfg.data.test = data_config['test']
model.cfg.model.roi_head.bbox_head.num_classes = len(data_config['train']['classes'])

# Set training parameters
model.cfg.optimizer.lr = 0.001  # Learning rate
model.cfg.data.samples_per_gpu = 2  # Batch size

# Build dataset and start training
datasets = [build_dataset(model.cfg.data.train)]
model.CLASSES = datasets[0].CLASSES

# Train the model
train_detector(model, datasets[0], cfg=model.cfg, distributed=False, validate=True)

In [ ]:
# Load the trained model for inference
checkpoint_file = '/content/drive/MyDrive/Count_AI_task/Aquarium_Combined.v2-raw-1024.coco-mmdetection/checkpoint.pth'  # Update with actual checkpoint path
model = init_detector(config_file, checkpoint_file, device='cuda:0')

# Run inference on test images
test_images_dir = os.path.join(dataset_root, 'test/images/')
for img_name in os.listdir(test_images_dir):
    img_path = os.path.join(test_images_dir, img_name)
    result = inference_detector(model, img_path)

    # Visualize results (you can save or display them)
    show_result(img_path, result, model.CLASSES)